In [62]:

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import time

In [63]:
import pandas as pd
import glob
import os

# Path to all CSV files
folder_path = "/kaggle/input/stock-market-data-nifty-100-stocks-15-min-data/*.csv"

# Dictionary to hold DataFrames
dfs_dict = {}

# Loop over CSV files
for file in glob.glob(folder_path):
    ticker = os.path.basename(file).replace(".csv", "")
    
    df = pd.read_csv(file, parse_dates=['date'])
    
    # Ensure chronological order
    df.sort_values('date', inplace=True)
    
    # Standardize column names
    df.columns = df.columns.str.lower()
    df.rename(columns={
        'open': 'open',
        'high': 'high',
        'low': 'low',
        'close': 'close',
        'volume': 'volume'
    }, inplace=True)
    
    # Set datetime index for time-series operations
    df.set_index('date', inplace=True)
    
    # Store in dictionary
    dfs_dict[ticker] = df

print("Loaded tickers:", list(dfs_dict.keys())[:10])  # first 10 tickers

Loaded tickers: ['IRFC_15minute', 'ADANIENSOL_15minute', 'ASIANPAINT_15minute', 'HCLTECH_15minute', 'DRREDDY_15minute', 'BAJAJHLDNG_15minute', 'TRENT_15minute', 'TITAN_15minute', 'ONGC_15minute', 'ABB_15minute']


In [64]:
# Define start and end dates
start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2025-12-31')

# Filter each DataFrame in dfs_dict
for ticker, df in dfs_dict.items():
    dfs_dict[ticker] = df[(df.index >= start_date) & (df.index <= end_date)]

# Verify
for ticker, df in list(dfs_dict.items())[:5]:
    print(f"{ticker}: {df.index.min()} to {df.index.max()}")


IRFC_15minute: 2022-01-03 09:15:00 to 2025-08-06 18:00:00
ADANIENSOL_15minute: 2022-01-03 09:15:00 to 2025-08-06 18:00:00
ASIANPAINT_15minute: 2022-01-03 09:15:00 to 2025-08-06 18:00:00
HCLTECH_15minute: 2022-01-03 09:15:00 to 2025-08-06 18:00:00
DRREDDY_15minute: 2022-01-03 09:15:00 to 2025-08-06 18:00:00


In [65]:
def daily_allocations(dfs_dict, selected):
    rows = []
    for t in selected:
        df = dfs_dict[t]
        df['ATR'] = ATR(df, period=14)
        last = df.iloc[-1]
        atr_pct = last['ATR'] / (last['close'] + 1e-9)
        rows.append({'Ticker': t, 'ATR_pct': atr_pct})
    alloc_df = pd.DataFrame(rows).set_index('Ticker')
    alloc_df['inv_vol'] = 1 / (alloc_df['ATR_pct'] + 1e-9)
    alloc_df['weight'] = alloc_df['inv_vol'] / alloc_df['inv_vol'].sum()
    alloc_df['allocation_INR'] = alloc_df['weight'] * 100000
    return alloc_df

In [66]:
import numpy as np
import pandas as pd

# ----------------- ATR Function -----------------
def ATR(df, period=14, method='SMA'):
    """
    Compute Average True Range (ATR) for a DataFrame with 'high', 'low', 'close' columns.
    
    Parameters:
        df (pd.DataFrame): OHLC data
        period (int): Lookback period for ATR
        method (str): 'SMA' or 'EMA'
    
    Returns:
        pd.Series: ATR values
    """
    hl = df['high'] - df['low']
    hc = (df['high'] - df['close'].shift()).abs()
    lc = (df['low'] - df['close'].shift()).abs()
    tr = pd.concat([hl, hc, lc], axis=1).max(axis=1)
    
    if method.upper() == 'EMA':
        return tr.ewm(span=period, adjust=False).mean()
    else:  # SMA
        return tr.rolling(period).mean()

# ----------------- RSI Function -----------------
def compute_rsi(series, period=14):
    """
    Compute Relative Strength Index (RSI) for a price series.
    """
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period, min_periods=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=period).mean()
    rs = gain / (loss + 1e-9)  # avoid division by zero
    return 100 - (100 / (1 + rs))

# ----------------- Add Indicators to All Tickers -----------------
dfs_indicators = {}
for ticker, df in dfs_dict.items():
    df = df.copy()
    
    # Moving Averages
    df["SMA_20"] = df["close"].rolling(20, min_periods=20).mean()
    df["SMA_50"] = df["close"].rolling(50, min_periods=50).mean()
    
    # Momentum Indicator
    df["RSI_14"] = compute_rsi(df["close"], 14)
    
    dfs_indicators[ticker] = df

# Combine all tickers into a single DataFrame for easy preview
df_all_indicators = pd.concat(
    dfs_indicators.values(),
    keys=dfs_indicators.keys(),
    names=["Ticker", "date"]
).reset_index(level=0)

print("Indicators added for all tickers. Preview:")
print(df_all_indicators.head())

# ----------------- Hurst Exponent -----------------
def hurst_exponent(ts, max_lag=20):
    """
    Estimate the Hurst exponent of a time series (trend vs mean-reversion).
    """
    ts = np.array(ts, dtype=float)
    if len(ts) < 2:
        return np.nan
    
    lags = range(2, min(max_lag, len(ts)))
    tau = [np.std(ts[lag:] - ts[:-lag]) for lag in lags]
    
    if any(t == 0 for t in tau):
        return np.nan
    
    slope = np.polyfit(np.log(lags), np.log(tau), 1)[0]
    H = slope * 2.0
    return H

# ----------------- VWAP Function -----------------
def vwap(df, price_col='close', volume_col='volume'):
    """
    Compute VWAP per trading day.
    """
    df = df.copy()
    
    # Ensure 'trade_date' column exists
    if isinstance(df.index, pd.DatetimeIndex):
        df['trade_date'] = df.index.date
    else:
        df['trade_date'] = pd.to_datetime(df['date']).dt.date
    
    vwap_values = df.groupby('trade_date').apply(
        lambda x: (x[price_col] * x[volume_col]).cumsum() / x[volume_col].cumsum()
    ).reset_index(level=0, drop=True)
    
    return vwap_values

# ----------------- Sortino Ratio -----------------
def sortino_ratio(returns, rf=0.0):
    """
    Compute Sortino ratio for a returns series.
    """
    excess = returns - rf
    downside = excess[excess < 0]
    if len(downside) == 0:
        return np.nan
    
    downside_dev = np.sqrt((downside**2).mean())
    mean_excess = excess.mean()
    return mean_excess / downside_dev

# ----------------- Market Regime -----------------
def compute_regime(close_series, short_window=20, long_window=50, threshold=0.002):
    """
    Classify market regime: Bullish, Bearish, Sideways
    """
    short_ma = close_series.rolling(short_window, min_periods=1).mean()
    long_ma = close_series.rolling(long_window, min_periods=1).mean()
    diff = (short_ma - long_ma) / long_ma

    regime = pd.Series("Sideways", index=close_series.index)
    regime[diff > threshold] = "Bullish"
    regime[diff < -threshold] = "Bearish"
    regime[diff.isna()] = "Unknown"

    return regime

# ----------------- ADX Function -----------------
def adx(df, period=14):
    high = df['high']
    low = df['low']
    close = df['close']
    tr = ATR(df, period)
    
    up = high - high.shift()
    down = low.shift() - low
    plus_dm = np.where((up > down) & (up > 0), up, 0)
    minus_dm = np.where((down > up) & (down > 0), down, 0)
    
    plus_di = 100 * pd.Series(plus_dm).rolling(period).mean() / tr
    minus_di = 100 * pd.Series(minus_dm).rolling(period).mean() / tr
    
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di + 1e-9)
    return dx.rolling(period).mean()


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: Ru

Indicators added for all tickers. Preview:
                            Ticker   open   high    low  close  volume  \
date                                                                     
2022-01-03 09:15:00  IRFC_15minute  22.95  23.05  22.90  23.00  504659   
2022-01-03 09:30:00  IRFC_15minute  22.95  23.10  22.95  23.05  598779   
2022-01-03 09:45:00  IRFC_15minute  23.05  23.05  23.00  23.05  195517   
2022-01-03 10:00:00  IRFC_15minute  23.05  23.05  22.95  23.00  404983   
2022-01-03 10:15:00  IRFC_15minute  23.00  23.00  22.90  22.95  376801   

                     SMA_20  SMA_50  RSI_14  
date                                         
2022-01-03 09:15:00     NaN     NaN     NaN  
2022-01-03 09:30:00     NaN     NaN     NaN  
2022-01-03 09:45:00     NaN     NaN     NaN  
2022-01-03 10:00:00     NaN     NaN     NaN  
2022-01-03 10:15:00     NaN     NaN     NaN  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [67]:
# ----------------- Apply Market Regime -----------------
print("Applying market regime to all tickers...\n")

for ticker, df in dfs_dict.items():
    df = df.copy()  # avoid modifying original DataFrame
    df["Regime"] = compute_regime(df["close"])  # compute regime
    dfs_dict[ticker] = df  # update dictionary
    
    # Print last 3 rows for quick verification
    print(f"Ticker: {ticker}")
    print(df[["close", "Regime"]].tail(3))
    print("-" * 40)


Applying market regime to all tickers...

Ticker: IRFC_15minute
                      close   Regime
date                                
2025-08-06 17:15:00  127.52  Bearish
2025-08-06 17:45:00  127.52  Bearish
2025-08-06 18:00:00  127.52  Bearish
----------------------------------------
Ticker: ADANIENSOL_15minute
                      close    Regime
date                                 
2025-08-06 17:15:00  794.95  Sideways
2025-08-06 17:45:00  794.95  Sideways
2025-08-06 18:00:00  794.95  Sideways
----------------------------------------
Ticker: ASIANPAINT_15minute
                      close   Regime
date                                
2025-08-06 17:15:00  2491.2  Bullish
2025-08-06 17:45:00  2491.2  Bullish
2025-08-06 18:00:00  2491.2  Bullish
----------------------------------------
Ticker: HCLTECH_15minute
                      close   Regime
date                                
2025-08-06 17:15:00  1463.2  Bearish
2025-08-06 17:45:00  1463.2  Bearish
2025-08-06 18:00:00  146

In [ ]:
import pandas as pd
import glob

# ----------------- Load CSVs and Compute Indicators -----------------
folder_path = "/kaggle/input/stock-market-data-nifty-100-stocks-15-min-data/*.csv"
dfs_indicators = {}

for file in glob.glob(folder_path):
    # Extract ticker name from filename
    ticker = file.split("/")[-1].replace("_15minute.csv", "")
    
    # Load CSV
    df = pd.read_csv(file, parse_dates=["date"])
    
    # Standardize column names
    df.columns = df.columns.str.lower()
    
    # Ensure chronological order and set index
    df = df.sort_values("date").set_index("date")
    
    # Compute indicators using previously defined compute_rsi function
    df["sma_20"] = df["close"].rolling(20, min_periods=20).mean()
    df["sma_50"] = df["close"].rolling(50, min_periods=50).mean()
    df["rsi_14"] = compute_rsi(df["close"], 14)
    
    # Store in dictionary
    dfs_indicators[ticker] = df

# Combine all tickers into a multi-index DataFrame
df_all_indicators = pd.concat(dfs_indicators, names=["Ticker", "date"])

# Preview results
print(f"Indicators added for {len(dfs_indicators)} tickers. Preview:")
print(df_all_indicators.tail())


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: Ru

In [ ]:
import pandas as pd
import glob
import os

# Path to all 15-minute CSV files
folder_path = "/kaggle/input/stock-market-data-nifty-100-stocks-15-min-data/*.csv"

# Load all CSVs into a single multi-index DataFrame
dfs_dict = {}

for file in glob.glob(folder_path):
    ticker = os.path.basename(file).replace("_15minute.csv", "")
    df = pd.read_csv(file, parse_dates=["date"])
    
    # Standardize column names
    df.columns = df.columns.str.lower()
    
    # Sort and set index
    df = df.sort_values("date").set_index("date")
    
    dfs_dict[ticker] = df

# Combine into a multi-index DataFrame if needed
df_all = pd.concat(dfs_dict, names=["Ticker", "date"])

print(f"Combined data for {len(dfs_dict)} tickers. Preview:")
print(df_all.tail(3))

# Example: loop through tickers for indicator calculation
for ticker, df in dfs_dict.items():
    print(f"Processing {ticker} with {df.shape[0]} rows")
    df = df.copy()
    # Call your indicator/strategy functions here


In [ ]:
# ----------------- Portfolio Aggregation -----------------
def aggregate_portfolio(per_stock_results, trades_df, capital=100000, risk_free_rate=0.052):
    """
    Aggregate P&L, equity curve, and risk metrics for intraday portfolio.

    Parameters:
        per_stock_results : placeholder for per-ticker results (not used directly)
        trades_df (pd.DataFrame): Must contain 'ExitTime' and 'P&L_INR'
        capital (float): Starting portfolio capital
        risk_free_rate (float): Annual risk-free rate

    Returns:
        trades_df, daily_pnl, daily_returns, equity_curve, drawdown, metrics
    """
    if trades_df.empty:
        return None

    trades_df['ExitTime'] = pd.to_datetime(trades_df['ExitTime'])

    # --- Daily P&L and returns ---
    daily_pnl = trades_df.set_index('ExitTime')['P&L_INR'].resample('D').sum().fillna(0)
    daily_returns = daily_pnl / capital

    # --- Equity curve ---
    equity_curve = (1 + daily_returns).cumprod() * capital

    # --- Drawdown ---
    drawdown = (equity_curve - equity_curve.cummax()) / equity_curve.cummax()
    max_dd = drawdown.min()

    # --- Annualization ---
    trading_days = 252
    daily_rf = (1 + risk_free_rate) ** (1 / trading_days) - 1  # daily risk-free rate

    # --- Sharpe ratio (annualized) ---
    avg_daily_return = daily_returns.mean()
    volatility = daily_returns.std() * (trading_days ** 0.5)
    sharpe = ((avg_daily_return - daily_rf) * trading_days) / volatility if volatility > 0 else float('nan')

    # --- Sortino ratio ---
    sortino = sortino_ratio(daily_returns, rf=daily_rf)  # uses previously defined function

    # --- ROI ---
    total_return = (equity_curve.iloc[-1] / capital) - 1
    cumulative_return = total_return * 100

    metrics = {
        'Cumulative_Return_%': cumulative_return,
        'ROI_%': cumulative_return,
        'Volatility_%': volatility * 100,
        'Max_Drawdown_%': max_dd * 100,
        'Sharpe_Ratio': sharpe,
        'Sortino_Ratio': sortino,
        'Total_Trades': len(trades_df),
        'Avg_PnL_per_Trade': trades_df['P&L_INR'].mean()
    }

    return trades_df, daily_pnl, daily_returns, equity_curve, drawdown, metrics


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

def daily_stock_selection(dfs_dict, top_n=10, max_depth=4):
    """
    Daily stock selection using Decision Tree based on trend & momentum features.

    Parameters:
        dfs_dict: dict {ticker: df with columns ['date', 'close', 'high', 'low', 'volume']}
        top_n: int - number of BUY and SELL stocks to return
        max_depth: int - depth of the decision tree

    Returns:
        dict with keys {'BUY': [tickers], 'SELL': [tickers]}
    """
    results = []

    for ticker, df in dfs_dict.items():
        df = df.copy().sort_values('date').reset_index(drop=True)
        if len(df) < 50:
            continue  # insufficient data

        # --- Compute Features ---
        df['EMA10'] = df['close'].ewm(span=10, adjust=False).mean()
        df['EMA30'] = df['close'].ewm(span=30, adjust=False).mean()
        df['RSI14'] = compute_rsi(df['close'], 14)
        df['VWAP'] = vwap(df)
        df['STD20'] = df['close'].rolling(20).std()
        df['VolumeAvg20'] = df['volume'].rolling(20).mean()
        df['VolumeRatio'] = df['volume'] / (df['VolumeAvg20'] + 1e-9)

        # --- Target: next day return sign ---
        df['FutureReturn'] = df['close'].shift(-1) - df['close']
        df['Target'] = 0
        df.loc[df['FutureReturn'] > 0, 'Target'] = 1
        df.loc[df['FutureReturn'] < 0, 'Target'] = -1

        # --- Feature Matrix ---
        features = ['EMA10', 'EMA30', 'RSI14', 'VWAP', 'STD20', 'VolumeRatio']
        df.dropna(subset=features + ['Target'], inplace=True)
        if len(df) < 30:
            continue  # not enough clean data

        X = df[features]
        y = df['Target']

        # --- Train Decision Tree ---
        clf = DecisionTreeClassifier(max_depth=max_depth)
        clf.fit(X, y)

        # --- Predict latest day ---
        latest = df.iloc[-1:][features]
        predicted_signal = clf.predict(latest)[0]
        proba = clf.predict_proba(latest)

        # Probabilities
        prob_buy = proba[0][list(clf.classes_).index(1)] if 1 in clf.classes_ else 0
        prob_sell = proba[0][list(clf.classes_).index(-1)] if -1 in clf.classes_ else 0

        # Append result
        if predicted_signal == 1:
            results.append({'Ticker': ticker, 'Direction': 'BUY', 'Score': prob_buy})
        elif predicted_signal == -1:
            results.append({'Ticker': ticker, 'Direction': 'SELL', 'Score': prob_sell})

    # --- Rank Top N BUY and SELL ---
    if not results:
        return {'BUY': [], 'SELL': []}

    res_df = pd.DataFrame(results)

    buy_candidates = (
        res_df[res_df['Direction'] == 'BUY']
        .sort_values('Score', ascending=False)
        .head(top_n)['Ticker']
        .tolist()
    )

    sell_candidates = (
        res_df[res_df['Direction'] == 'SELL']
        .sort_values('Score', ascending=False)
        .head(top_n)['Ticker']
        .tolist()
    )

    return {'BUY': buy_candidates, 'SELL': sell_candidates}


In [ ]:
from datetime import time
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# --- Constants ---
TRADE_START_TIME = time(9, 30)
SQUARE_OFF_TIME = time(15, 30)
TRADING_DAYS = 252

# --- Improved Decision Tree Signal Generation with Rolling Retraining ---
def generate_signals_rolling(dfs_dict, lookback=504, forward_days=3, return_threshold=0.005,
                             max_depth=3, min_samples_leaf=20):
    """
    Generate signals per ticker using rolling retraining to reduce overfitting.
    """
    dfs_signals = {}
    hurst_values = {}

    for ticker, df in dfs_dict.items():
        df = df.copy().sort_values('date').reset_index(drop=True)
        if len(df) < lookback + forward_days:
            continue

        # Features
        df['VWAP'] = vwap(df)
        df['EMA5'] = df['close'].ewm(span=5, adjust=False).mean()
        df['EMA20'] = df['close'].ewm(span=20, adjust=False).mean()
        df['RSI14'] = compute_rsi(df['close'], 14)
        df['RSI5'] = compute_rsi(df['close'], 5)
        df['STD20'] = df['close'].rolling(20).std()
        df['ATR14'] = ATR(df, 14)
        df['Hurst'] = np.nan
        if len(df) >= lookback:
            df['Hurst'].iloc[-1] = hurst_exponent(df['close'].values[-lookback:])
            hurst_values[ticker] = df['Hurst'].iloc[-1]
        else:
            hurst_values[ticker] = np.nan

        # Forward returns
        df['FutureReturn'] = df['close'].shift(-forward_days) / df['close'] - 1
        df['SignalTarget'] = 0
        df.loc[df['FutureReturn'] > return_threshold, 'SignalTarget'] = 1
        df.loc[df['FutureReturn'] < -return_threshold, 'SignalTarget'] = -1

        # Features and target
        features = ['VWAP', 'EMA5', 'EMA20', 'RSI14', 'RSI5', 'STD20', 'ATR14', 'Hurst']
        df.dropna(subset=features + ['SignalTarget'], inplace=True)

        if len(df) < 50 or len(df['SignalTarget'].unique()) < 2:
            continue

        # --- Rolling training ---
        split_idx = len(df) - forward_days
        X_train, y_train = df[features].iloc[:split_idx], df['SignalTarget'].iloc[:split_idx]
        X_test = df[features].iloc[split_idx:]
        df.loc[X_test.index, 'Signal'] = 0

        if len(np.unique(y_train)) >= 2:
            clf = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
            clf.fit(X_train, y_train)
            df.loc[X_test.index, 'Signal'] = clf.predict(X_test)
            # store prediction probabilities for confidence-based allocation
            proba = clf.predict_proba(X_test)
            df.loc[X_test.index, 'SignalConfidence'] = np.max(proba, axis=1)

        # --- Market regime filter ---
        df['Signal'] = df['Signal'].where(df.get('Regime', 'Sideways') == 'Bullish', 0)
        df['Signal'] = df['Signal'].where(df.get('Regime', 'Sideways') == 'Bearish', 0)
        # --- Volatility filter ---
        df['Volatility'] = df['STD20'] / df['close']
        df.loc[df['Volatility'] < 0.008, 'Signal'] = 0

        dfs_signals[ticker] = df

    return dfs_signals, hurst_values


# --- Improved Backtesting with Adaptive Stop-loss & Confidence-based Position Sizing ---
def backtest_intraday_improved(dfs_dict, selected, allocations, market_regime="Neutral",
                               atr_multiplier=1.0, max_position_fraction=0.2):
    """
    Multi-ticker intraday backtest with adaptive stop-loss and position scaling.
    """
    trade_logs = []
    per_stock_results = {}

    for tick in selected:
        if tick not in dfs_dict or tick not in allocations.index:
            continue

        df = dfs_dict[tick].copy()
        if 'ATR14' not in df.columns:
            df['ATR14'] = ATR(df, period=14)
        df = df[df['close'] > 0].dropna(subset=['close', 'ATR14']).sort_values('date').reset_index(drop=True)

        position = 0
        entry_price = 0.0
        entry_idx = None
        allocation = allocations.loc[tick, 'allocation_INR']
        quantity = 0

        for i, row in df.iterrows():
            ttime = row['date'].time() if 'date' in row else row['Datetime'].time()
            signal = row.get('Signal', 0)
            price = row['close']
            atr = row['ATR14']
            confidence = row.get('SignalConfidence', 1.0)

            if price <= 0 or pd.isna(price) or pd.isna(atr):
                continue

            # --- Adaptive position sizing based on confidence ---
            quantity = max(1, int(allocation * min(confidence, max_position_fraction) // price))

            # --- Entry ---
            if position == 0 and signal != 0 and TRADE_START_TIME <= ttime < SQUARE_OFF_TIME:
                position = signal
                entry_price = price
                entry_idx = i

            # --- Adaptive Exit ---
            exit_flag, exit_reason = False, None
            if position != 0:
                sl_price = entry_price - position * atr_multiplier * atr
                if (position == 1 and price <= sl_price) or (position == -1 and price >= sl_price):
                    exit_flag, exit_reason = True, 'SL_hit'
                if signal == -position and ttime < SQUARE_OFF_TIME:
                    exit_flag, exit_reason = True, 'Signal_flip'
                if ttime >= SQUARE_OFF_TIME:
                    exit_flag, exit_reason = True, 'Square_off'

            if exit_flag and position != 0:
                exit_price = price
                pnl = (exit_price - entry_price) * quantity * position
                trade_logs.append({
                    'Ticker': tick,
                    'EntryTime': df.loc[entry_idx, 'date'],
                    'ExitTime': row['date'],
                    'Side': 'Long' if position == 1 else 'Short',
                    'EntryPrice': entry_price,
                    'ExitPrice': exit_price,
                    'Quantity': quantity,
                    'P&L_INR': pnl,
                    'Reason': exit_reason
                })
                position = 0
                entry_price = 0.0
                entry_idx = None

            df.at[i, 'Position'] = position

        per_stock_results[tick] = {'df': df, 'allocation': allocation}

    trades_df = pd.DataFrame(trade_logs)
    return per_stock_results, trades_df


In [ ]:
if __name__ == "__main__":
    import glob
    import pandas as pd

    # --- Load CSVs ---
    folder_path = "/kaggle/input/stock-market-data-nifty-100-stocks-15-min-data/*.csv"
    dfs = {}
    for file in glob.glob(folder_path):
        df = pd.read_csv(file, parse_dates=["date"]).sort_values("date").reset_index(drop=True)
        ticker = file.split("/")[-1].replace("_15minute.csv", "")
        df["Ticker"] = ticker
        dfs[ticker] = df
    print(f"Loaded {len(dfs)} tickers.")

    # --- Compute Market Regime ---
    try:
        first_ticker = list(dfs.keys())[0]
        market_regime_series = compute_regime(dfs[first_ticker]["close"])
        market_regime = market_regime_series.iloc[-1]
        print("Market Regime:", market_regime)
    except Exception as e:
        print("Failed to compute regime, defaulting to Sideways. Error:", e)
        market_regime = "Sideways"

    # --- Daily Stock Selection ---
    TOP_N = 5
    selected_dict = daily_stock_selection(dfs, top_n=TOP_N)
    selected_today = selected_dict['BUY'] + selected_dict['SELL']
    print("Selected tickers today:", selected_today)

    if len(selected_today) == 0:
        print("No tickers selected. Exiting.")
    else:
        # --- Allocations ---
        alloc_df = daily_allocations(dfs, selected_today)
        print("\nAllocations (INR) per selected ticker:")
        print(alloc_df[['ATR_pct', 'weight', 'allocation_INR']])

        # --- Backtest with Train/Test Split ---
        print("\nRunning intraday backtest with train/test split...")
        train_results, train_trades_df, test_results, test_trades_df = backtest_intraday_train_test(
            dfs, selected_today, alloc_df, market_regime
        )

        # --- Aggregate Portfolio Metrics ---
        print("\n--- Training Metrics ---")
        if train_trades_df is not None and not train_trades_df.empty:
            agg_train = aggregate_portfolio(train_results, train_trades_df)
            if agg_train:
                _, _, _, _, _, metrics_train = agg_train
                for k, v in metrics_train.items():
                    print(f"{k}: {v}")
        else:
            print("No trades executed in training period.")

        print("\n--- Testing Metrics ---")
        if test_trades_df is not None and not test_trades_df.empty:
            agg_test = aggregate_portfolio(test_results, test_trades_df)
            if agg_test:
                _, _, _, _, _, metrics_test = agg_test
                for k, v in metrics_test.items():
                    print(f"{k}: {v}")
        else:
            print("No trades executed in testing period.")
